In [1]:
#Sushant ...the code to take an input folder containing multiple PDFs,
#merge them into one PDF, and then convert the merged PDF into PNG format
import tkinter as tk
from tkinter import filedialog
from PyPDF2 import PdfMerger
import fitz
import os

# Function to merge PDF files from a folder
def merge_pdfs_from_folder(input_folder, output_merged_pdf):
    pdf_merger = PdfMerger()

    pdf_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.lower().endswith('.pdf')]

    if not pdf_files:
        result_label.config(text="No PDF files found in the folder.")
        return

    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)

    try:
        with open(output_merged_pdf, 'wb') as output:
            pdf_merger.write(output)
            result_label.config(text="PDFs merged to: " + output_merged_pdf)
    except Exception as e:
        result_label.config(text="Error: " + str(e))

# Function to convert a merged PDF to PNGs with 600 DPI
def pdf_to_png(merged_pdf, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    pdf_document = fitz.open(merged_pdf)
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        image = page.get_pixmap(matrix=fitz.Matrix(600/72, 600/72))  # Set xres and yres to 600 DPI
        image.save(os.path.join(output_folder, f'page_{page_num + 1}.png'))
    pdf_document.close()

# Function to handle the PDF merging and conversion
def merge_and_convert_pdf():
    input_folder = filedialog.askdirectory(title="Select Folder with PDFs to Merge")
    if input_folder:
        output_merged_pdf = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])
        if output_merged_pdf:
            merge_pdfs_from_folder(input_folder, output_merged_pdf)
            result_label.config(text="PDFs merged to: " + output_merged_pdf)

            output_folder = filedialog.askdirectory(title="Select Output Folder for PNGs")
            if output_folder:
                pdf_to_png(output_merged_pdf, output_folder)
                result_label.config(text="PDF converted to PNGs in: " + output_folder)

app = tk.Tk()
app.title("PDF Merger and Converter")

# Create a button for merging and converting
merge_convert_button = tk.Button(app, text="Merge PDFs and Convert to PNG", command=merge_and_convert_pdf)
merge_convert_button.pack(padx=20, pady=20)

result_label = tk.Label(app, text="")
result_label.pack(pady=10)

app.mainloop()
